In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from newspaper import Article
import time
import random
import os
from newspaper import Config

In [2]:
def make_urllist_sub(index, date, page):
    sub_urllist = []
#     user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.90 Safari/537.36'
#     config = Config()
#     config.browser_user_agent = user_agent
#     config.request_timeout = 10
    url = 'https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1='+str(index)+'&date='+str(date)+'&page='+str(page)
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.90 Safari/537.36'}
    news = requests.get(url, headers=headers)
    soup = BeautifulSoup(news.content, 'html.parser')
    news_list = soup.select('.newsflash_body .type06_headline li dl')
    news_list.extend(soup.select('.newsflash_body .type06 li dl'))
    for line in news_list:
        sub_urllist.append(line.a.get('href'))
    return sub_urllist

In [3]:
def make_urllist(index, date):
    total_url_list = []
    temp_url = []
    for page in range(1, 1000):
        url_list = make_urllist_sub(index, date, page)
        if url_list[-1] == temp_url:
            break
        temp_url = url_list[-1]
        total_url_list.extend(url_list)
    return total_url_list

In [4]:
def make_data(urllist, index, date):
    title_list = []
    text_list = []
    item_list = []
    for url in urllist:
        temp_sen = url.split('=')
        item_list.append(temp_sen[-1])
        article = Article(url, language='ko')
        article.download()
        article.parse()
        title_list.append(article.title)
        text_list.append(article.text)
    df = pd.DataFrame({'title': title_list, 'permalink': urllist, 'item_id': item_list, 'contents': text_list, 'section_id': index, 'datetime':date})
    return df

In [6]:
def make_day_data(index, date):
    t = time.time()
    url_list = make_urllist(index, date)
    df = make_data(url_list, index, date)
    print(time.time()-t)
    time.sleep(random.uniform(5, 10))
    return df.to_csv(str(date) + '.csv')

In [7]:
def start(index, startday):
    for date in range(startday, startday + 31, 1): #31 부분만 수정하고 돌리시면 됩니다.
        make_day_data(index, date)
        time.sleep(random.uniform(5, 10))

## index

### 100 : 정치
### 101 : 경제
### 102 : 사회
### 103 : 생활/문화
### 104 : 세계
### 105 : IT/과학
### 110 : 오피니

# 파라미터는 index와 startday만 변경해주시면 됩니다.
# 더 필요하시면 make_total_data의 주석을 참고해주세요

In [8]:
index = 100 #인덱스 설정은 개인별로 바꾸시길 바랍니다.

In [9]:
startday = 20201001 # 작업 시작일입니다.

In [ ]:
start(index, startday)